**Fin 585R**  
**Diether**  
**Problem Set**  
**Analyst Dispersion Portfolios**  

**Purpose/Goal**

The primary purpose of this problem set is to give you a portfolio formation task that makes you go through the first four steps of our portfolio formation framework.

1. Data Preparation.<br><br>

2. Create portfolio formation or criterion variable.<br><br>

3. Bin the data based on the formation variable.<br><br>

4. Portfolio creeation using the bins.<br><br>

5. Test the historical performance and/or test a model.<br><br>

A secondary goal is to introduce another interesting portfolio strategy. It's produces a large spread in average return, and we will use it later in testing models like the CAPM.

To accomplish the programming takes, you should be able to adapt a lot of code we've used before, and apply it this situation. 

**Overview**

In this problem set you reproduce another seminal empirical result in academic finance. Specifically, you reproduce the **dispersion effect** (or the analyst disgreement effect) of Diether, Malloy, and Scherbina (2002). This empirical result spawned a large literature in academic finance, and certainly some quant funds have tried to trade on this effect.

Dispersion (or analyst disagreement) portfolios are formed based on the standard deviation of analyst eps (earnings per share) forecasts over a given period. Here the standard deviation of analyst eps forecasts is the standard deviation across analysts for a given stock and month (most stocks have between 3 to 13 analysts covering them). Diether, Malloy, and Scherbina don't use raw standard deviation. Instead, they scale the standard deviation of analyst forecasts by the absolute value of the mean forecast. Therefore for a given month ($t$), dispersion for stock $i$ is defined as the following:

$$
disp_{it} = \frac{stdev_{it}}{|mean_{it}|}
$$

DMS form dispersion portfolios using $disp_{i,t-1}$; in other words, they lag dispersion one month. In this homework you will do the same. Additionally, you will form dispersion portfolio based on lagging dispersion 3 months.

There are two datasets for this problem set. The first is the CRSP data (security prices and returns) during the period from January of 1982 to December of 2000. The second is the analyst earnings per share data from IBES. It also covers the period of January of 1982 to December of 2000. The frequency for both datasets is monthly. The stock level identifier in the IBES data is called a CUSIP. Consequently, I also included CUSIPs in the CRSP data. The CUSIP and the calender month uniquely identify the analyst earnings per share observations.

You can download the CRSP data directly using the following link: [the CRSP data](http://diether.org/prephd/08-mstk_82-00.csv). There is also a link on *Learning Suite*. The data contain the following variables:

|Variable | Description                                              |
|---------|----------------------------------------------------------|
|permno   | stock identifier                                         |
|cusip    | stock identifier also in IBES data                       |
|caldt    | calendar date (the day is not truncated to 1)            |
|ret      | monthly return                                           |
|prc      | stock price (not lagged, contemporaneous with returns)   |   


You can download the IBES data directly using the following link: [the IBES data](http://diether.org/prephd/08-ibes_eps_analyst.csv). There is also a link on *Learning Suite*. The data contain the following variables:

|Variable | Description                                          |
|---------|------------------------------------------------------|
|cusip    | stock identifier also in IBES data                   |
|caldt    | calendar date (the day is not truncated to 1)        |
|meanest  | average analyst forecast for that month/stock        |
|stdev    | standard deviation of forecasts for that month/stock |


**Tasks**

1. Form quintile based equal-weight dispersion portfolios where dispersion is lagged one month. Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5). <br><br>

2. Add a spread portfolio to your dataframe of dispersion portfolios. Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio).<br><br>

3. Compute the average number of stocks that are in each portfolio.<br><br>

4. Form quintile based equal-weight dispersion portfolios where dispersion is lagged three month instead of one.  Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5).<br><br>

5. Compare the results from (1) and (4). What do either the differences or similarities in the average return pattern tell you about the nature of this dispersion effect?

In [1]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary

In [2]:
stk = pd.read_csv('../data/08-mstk_82-00.csv', parse_dates=['caldt'])
stk.head(5)

,permno,caldt,cusip,ret,prc,me
0,10000,1986-01-31,68391610,NaN,4.37500,16.1000
1,10000,1986-02-28,68391610,-0.257143,3.25000,11.9600
2,10000,1986-03-31,68391610,0.365385,4.43750,16.3300
3,10000,1986-04-30,68391610,-0.098592,4.00000,15.1720
4,10000,1986-05-30,68391610,-0.222656,3.10938,11.7939


In [3]:
ibes = pd.read_csv("../data/08-ibes_eps_analyst.csv", parse_dates=['caldt'])
ibes.head(5)

,cusip,caldt,meanest,stdev
0,00000117,1982-01-14,15.36,0.78
1,00000117,1982-02-18,15.18,0.78
2,00000117,1982-03-18,15.07,0.66
3,00000117,1982-04-15,15.06,0.70
4,00000117,1982-05-20,14.78,0.71


**Hint About Merging the two Datasets**

In the datasets I've include the full calender dates of the observations. Even though the frequency for both is monthly, the timing is not the same. The CRSP data is from the last trading day in the month, and the IBES data tends to be around the middle of the month. Therefore, to merge these dataframes you need to ctreate a new date variable that only preserve uniqueness at the year-month level. Here is a shortcut way to accomplish that:

In [4]:
stk['mdt'] = stk['caldt'].values.astype('datetime64[M]')
stk.head(5)

,permno,caldt,cusip,ret,prc,me,mdt
0,10000,1986-01-31,68391610,NaN,4.37500,16.1000,1986-01-01
1,10000,1986-02-28,68391610,-0.257143,3.25000,11.9600,1986-02-01
2,10000,1986-03-31,68391610,0.365385,4.43750,16.3300,1986-03-01
3,10000,1986-04-30,68391610,-0.098592,4.00000,15.1720,1986-04-01
4,10000,1986-05-30,68391610,-0.222656,3.10938,11.7939,1986-05-01


In [5]:
ibes['mdt'] = ibes['caldt'].values.astype('datetime64[M]')
ibes.head(5)

,cusip,caldt,meanest,stdev,mdt
0,00000117,1982-01-14,15.36,0.78,1982-01-01
1,00000117,1982-02-18,15.18,0.78,1982-02-01
2,00000117,1982-03-18,15.07,0.66,1982-03-01
3,00000117,1982-04-15,15.06,0.70,1982-04-01
4,00000117,1982-05-20,14.78,0.71,1982-05-01


What is the code above doing? Pandas stores all dates with precision to the nanosecond. But numpy (the library pandas uses for its date functionality) actually includes date types for varying levels of precision (including monthly). So the above code changes the original nanosecond datetype to a monthly datetype; this causes all the information about time beyond a month to be lost and when pandas automatically reconverts the date to a nanosecond datetype the day gets set equal to one for all observations.

Now you should be able to merge the two datasets.

In [6]:
df = stk.merge(ibes[['cusip', 'mdt', 'meanest', 'stdev']], on=['cusip', 'mdt'], how='left')
df.head()

,permno,caldt,cusip,ret,prc,me,mdt,meanest,stdev
0,10000,1986-01-31,68391610,NaN,4.37500,16.1000,1986-01-01,NaN,NaN
1,10000,1986-02-28,68391610,-0.257143,3.25000,11.9600,1986-02-01,NaN,NaN
2,10000,1986-03-31,68391610,0.365385,4.43750,16.3300,1986-03-01,NaN,NaN
3,10000,1986-04-30,68391610,-0.098592,4.00000,15.1720,1986-04-01,NaN,NaN
4,10000,1986-05-30,68391610,-0.222656,3.10938,11.7939,1986-05-01,NaN,NaN


### 1.
Form quintile based equal-weight dispersion portfolios where dispersion is lagged one month. Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5).

In [7]:
df['disp'] = df['stdev'] / df['meanest'].abs()
df['disp_lag'] = df.groupby('cusip')['disp'].shift()
df['prc_lag'] = df.groupby('cusip')['prc'].shift()
df.head()

,permno,caldt,cusip,ret,prc,me,mdt,meanest,stdev,disp,disp_lag,prc_lag
0,10000,1986-01-31,68391610,NaN,4.37500,16.1000,1986-01-01,NaN,NaN,NaN,NaN,NaN
1,10000,1986-02-28,68391610,-0.257143,3.25000,11.9600,1986-02-01,NaN,NaN,NaN,NaN,4.3750
2,10000,1986-03-31,68391610,0.365385,4.43750,16.3300,1986-03-01,NaN,NaN,NaN,NaN,3.2500
3,10000,1986-04-30,68391610,-0.098592,4.00000,15.1720,1986-04-01,NaN,NaN,NaN,NaN,4.4375
4,10000,1986-05-30,68391610,-0.222656,3.10938,11.7939,1986-05-01,NaN,NaN,NaN,NaN,4.0000


In [8]:
df = df.query("disp_lag == disp_lag and prc_lag >= 5").reset_index(drop=True)    # where price > 5
df['bins'] = df.groupby('mdt')['disp_lag'].transform(pd.qcut, 5, labels=False)
df.head()

,permno,caldt,cusip,ret,prc,me,mdt,meanest,stdev,disp,disp_lag,prc_lag,bins
0,10001,1990-05-31,39040610,-0.012658,9.75,10.0132,1990-05-01,1.05,0.07,0.066667,0.140000,9.875,3
1,10001,1990-06-29,39040610,0.014103,9.75,10.0523,1990-06-01,1.10,0.14,0.127273,0.066667,9.750,2
2,10001,1990-07-31,39040610,0.025641,10.00,10.3100,1990-07-01,1.10,0.14,0.127273,0.127273,9.750,3
3,10001,1990-08-31,39040610,-0.050000,9.50,9.7945,1990-08-01,1.05,0.08,0.076190,0.127273,10.000,3
4,10001,1990-09-28,39040610,0.040789,9.75,10.1790,1990-09-01,NaN,NaN,NaN,0.076190,9.500,3


In [9]:
port = df.groupby(['mdt','bins'])['ret'].mean()*100
port = port.unstack(level='bins')
summary(port).round(2)

bins,0,1,2,3,4
count,227.00,227.00,227.00,227.00,227.00
mean,1.60,1.44,1.34,1.22,0.82
std,4.78,4.85,5.18,5.62,6.46
tstat,5.03,4.48,3.90,3.27,1.91
pval,0.00,0.00,0.00,0.00,0.06
min,-25.76,-25.09,-26.94,-29.43,-32.22
25%,-1.21,-1.63,-1.92,-1.90,-2.63
50%,1.82,1.63,1.95,1.70,1.36
75%,4.61,4.64,4.85,4.80,4.48
max,13.21,13.17,14.28,13.95,19.53


### 2.
Add a spread portfolio to your dataframe of dispersion portfolios. Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio).

In [10]:
port['spread'] = port[0] - port[4]
summary(port).round(2)

bins,0,1,2,3,4,spread
count,227.00,227.00,227.00,227.00,227.00,227.00
mean,1.60,1.44,1.34,1.22,0.82,0.78
std,4.78,4.85,5.18,5.62,6.46,3.54
tstat,5.03,4.48,3.90,3.27,1.91,3.31
pval,0.00,0.00,0.00,0.00,0.06,0.00
min,-25.76,-25.09,-26.94,-29.43,-32.22,-19.66
25%,-1.21,-1.63,-1.92,-1.90,-2.63,-0.70
50%,1.82,1.63,1.95,1.70,1.36,0.85
75%,4.61,4.64,4.85,4.80,4.48,2.60
max,13.21,13.17,14.28,13.95,19.53,14.92


### 3.
Compute the average number of stocks that are in each portfolio.

In [11]:
count = df.groupby(['mdt','bins'])['permno'].count()
count = count.unstack(level='bins')
count

bins,0,1,2,3,4
mdt,,,,,
1982-02-01,307,307,306,307,307
1982-03-01,303,302,304,301,303
1982-04-01,303,303,303,303,303
1982-05-01,303,303,303,303,303
1982-06-01,308,307,307,307,308
...,...,...,...,...,...
2000-08-01,633,627,628,627,629
2000-09-01,631,632,633,628,631
2000-10-01,621,620,620,621,618


In [12]:
count.mean()

bins
0    516.973568
1    515.321586
2    515.127753
3    514.850220
4    515.171806
dtype: float64

### 4.
Form quintile based equal-weight dispersion portfolios where dispersion is lagged three month instead of one.  Report summary statistics (including a t-test of whether the average return is statistically different from zero for each portfolio). Note, you should exclude low price stocks from your portfolios (price below $5).

In [13]:
df['three_lag'] = df.groupby('cusip')['disp'].shift(3)
df.head()

,permno,caldt,cusip,ret,prc,me,mdt,meanest,stdev,disp,disp_lag,prc_lag,bins,three_lag
0,10001,1990-05-31,39040610,-0.012658,9.75,10.0132,1990-05-01,1.05,0.07,0.066667,0.140000,9.875,3,NaN
1,10001,1990-06-29,39040610,0.014103,9.75,10.0523,1990-06-01,1.10,0.14,0.127273,0.066667,9.750,2,NaN
2,10001,1990-07-31,39040610,0.025641,10.00,10.3100,1990-07-01,1.10,0.14,0.127273,0.127273,9.750,3,NaN
3,10001,1990-08-31,39040610,-0.050000,9.50,9.7945,1990-08-01,1.05,0.08,0.076190,0.127273,10.000,3,0.066667
4,10001,1990-09-28,39040610,0.040789,9.75,10.1790,1990-09-01,NaN,NaN,NaN,0.076190,9.500,3,0.127273


In [14]:
df = df.query("three_lag == three_lag and prc_lag >= 5").reset_index(drop=True)    # where price > 5
df['new_bins'] = df.groupby('mdt')['three_lag'].transform(pd.qcut, 5, labels=False)
df.head()

,permno,caldt,cusip,ret,prc,me,mdt,meanest,stdev,disp,disp_lag,prc_lag,bins,three_lag,new_bins
0,10001,1990-08-31,39040610,-0.050000,9.500,9.7945,1990-08-01,1.05,0.08,0.076190,0.127273,10.000,3,0.066667,2
1,10001,1990-09-28,39040610,0.040789,9.750,10.1790,1990-09-01,NaN,NaN,NaN,0.076190,9.500,3,0.127273,3
2,10001,1991-05-31,39040610,0.000000,9.875,10.4083,1991-05-01,1.13,0.04,0.035398,0.035398,9.875,1,0.127273,3
3,10001,1991-06-28,39040610,0.078481,10.500,11.2665,1991-06-01,1.13,0.03,0.026549,0.035398,9.875,1,0.076190,2
4,10001,1991-08-30,39040610,0.135802,11.500,12.3395,1991-08-01,1.12,0.03,0.026786,0.026549,10.125,1,0.035398,1


In [15]:
port = df.groupby(['mdt','new_bins'])['ret'].mean()*100
port = port.unstack(level='new_bins')
summary(port).round(2)

new_bins,0,1,2,3,4
count,224.00,224.00,224.00,224.00,224.00
mean,1.49,1.45,1.32,1.17,1.02
std,4.70,4.81,5.11,5.50,6.42
tstat,4.76,4.51,3.86,3.18,2.38
pval,0.00,0.00,0.00,0.00,0.02
min,-25.37,-24.75,-26.78,-29.20,-32.30
25%,-1.03,-1.67,-1.95,-1.93,-2.28
50%,1.68,1.93,1.82,1.62,1.51
75%,4.50,4.49,4.41,4.58,4.70
max,12.29,12.54,14.62,14.25,20.61


In [16]:
port['spread'] = port[0] - port[4]
summary(port).round(2)

new_bins,0,1,2,3,4,spread
count,224.00,224.00,224.00,224.00,224.00,224.00
mean,1.49,1.45,1.32,1.17,1.02,0.47
std,4.70,4.81,5.11,5.50,6.42,3.55
tstat,4.76,4.51,3.86,3.18,2.38,2.00
pval,0.00,0.00,0.00,0.00,0.02,0.05
min,-25.37,-24.75,-26.78,-29.20,-32.30,-20.76
25%,-1.03,-1.67,-1.95,-1.93,-2.28,-1.22
50%,1.68,1.93,1.82,1.62,1.51,0.59
75%,4.50,4.49,4.41,4.58,4.70,2.10
max,12.29,12.54,14.62,14.25,20.61,12.67


### 5.
Compare the results from (1) and (4). What do either the differences or similarities in the average return pattern tell you about the nature of this dispersion effect?

The dispersion effect seems to last three months, but it's magnitude of effect and it's significance is greater in the first month. We can't be sure why, but this could be because analysts' reports are being priced in or because new analist reports are released.